In [48]:
import requests
import pandas as pd

def get_romania_life_expectancy():
    """Get Romania life expectancy data: Year, Sex, Life_Expectancy"""

    url = "https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/demo_mlexpec"
    params = {
        'format': 'json',
        'geo': 'RO',
        'age': 'Y_LT1',  # Life expectancy at birth
        'lang': 'en'
    }

    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()

    values = data['value']
    dimensions = data['dimension']

    # Get dimension sizes for key decoding
    dim_sizes = []
    dim_names = list(dimensions.keys())

    for dim_name in dim_names:
        size = len(dimensions[dim_name]['category']['index'])
        dim_sizes.append(size)

    # Get labels
    time_labels = dimensions['time']['category']['label']
    sex_labels = dimensions['sex']['category']['label']

    def decode_key(key_num, sizes):
        """Convert single number key to multi-dimensional indices"""
        key_num = int(key_num)
        indices = []
        for size in reversed(sizes):
            indices.append(key_num % size)
            key_num //= size
        return list(reversed(indices))

    # Extract data
    records = []
    for key_str, life_exp in values.items():
        if life_exp is not None:
            indices = decode_key(key_str, dim_sizes)

            # Get year and sex
            time_idx = indices[dim_names.index('time')]
            sex_idx = indices[dim_names.index('sex')]

            year = list(time_labels.values())[time_idx]
            sex = list(sex_labels.values())[sex_idx]

            records.append({
                'Year': int(year),
                'Sex': sex,
                'Life_Expectancy': life_exp
            })

    # Create and sort DataFrame
    df = pd.DataFrame(records)
    return df.sort_values(['Year', 'Sex']).reset_index(drop=True)

# Get data and display
df = get_romania_life_expectancy()

print("ROMANIA LIFE EXPECTANCY DATA")
print("="*40)
print(f"Records: {len(df)} | Years: {df['Year'].min()}-{df['Year'].max()}")
print(f"Categories: {list(df['Sex'].unique())}")
print("\nFirst 10 rows:")
print(df.head(10).to_string(index=False))


df.to_excel(r'C:\Users\Svitlana\OneDrive\RoFacts\mvp\data\life_expectancy.xlsx', index=False)



ROMANIA LIFE EXPECTANCY DATA
Records: 168 | Years: 1968-2023
Categories: ['Females', 'Males', 'Total']

First 10 rows:
 Year     Sex  Life_Expectancy
 1968 Females             70.3
 1968   Males             66.0
 1968   Total             68.2
 1969 Females             69.7
 1969   Males             65.1
 1969   Total             67.4
 1970 Females             70.4
 1970   Males             65.9
 1970   Total             68.2
 1971 Females             70.8


In [ ]:
import pandas as pd  # Needed to read the Excel file
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Load the saved data
file_path = r'C:\Users\Svitlana\OneDrive\RoFacts\mvp\data\life_expectancy.xlsx'
df = pd.read_excel(file_path)

# Define colors matching your reference
color_background = '#F8EFDE'
color_male = '#4f7d8f'  # Blue for males
color_female = '#c46e6e'  # Red for females
color_total = '#46C07a'  # Green for total (using your wage color)

def create_life_expectancy_chart():
    """Create life expectancy chart with built-in toggle button"""
    
    fig = go.Figure()
    
    # Prepare data
    male_data = df[df['Sex'] == 'Males']
    female_data = df[df['Sex'] == 'Females']
    total_data = df[df['Sex'] == 'Total']
    
    # Add all traces (we'll control visibility with buttons)
    if not male_data.empty:
        fig.add_trace(go.Scatter(
            x=male_data['Year'], 
            y=male_data['Life_Expectancy'],
            mode='lines+markers',
            name='Males',
            line=dict(color=color_male, width=3),
            marker=dict(size=6),
            visible=True
        ))
    
    if not female_data.empty:
        fig.add_trace(go.Scatter(
            x=female_data['Year'], 
            y=female_data['Life_Expectancy'],
            mode='lines+markers',
            name='Females',
            line=dict(color=color_female, width=3),
            marker=dict(size=6),
            visible=True
        ))
    
    if not total_data.empty:
        fig.add_trace(go.Scatter(
            x=total_data['Year'], 
            y=total_data['Life_Expectancy'],
            mode='lines+markers',
            name='Total',
            line=dict(color=color_total, width=3),
            marker=dict(size=6),
            visible=False
        ))
    
    # Update layout with your styling and buttons
    fig.update_layout(
        title='Romania Life Expectancy (1968-2023)',
        xaxis_title='Year',
        yaxis_title='Life Expectancy (Years)',
        plot_bgcolor=color_background,
        paper_bgcolor=color_background,
        xaxis=dict(
            dtick=2  # Show every 2 years
        ),
        yaxis=dict(
            tickformat='.1f'  # One decimal place
        ),
        legend=dict(
            x=0.02,
            y=0.98,
            bgcolor='rgba(255,255,255,0.8)',
            bordercolor='rgba(0,0,0,0.2)',
            borderwidth=1
        ),
        hovermode='x unified',
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=list([
                    dict(
                        args=[{"visible": [True, True, False]}],
                        label="By Gender",
                        method="restyle"
                    ),
                    dict(
                        args=[{"visible": [False, False, True]}],
                        label="Total Only",
                        method="restyle"
                    )
                ]),
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.5,
                xanchor="center",
                y=1.2,
                yanchor="middle",
                bgcolor='rgba(255,255,255,0.8)',
                bordercolor='rgba(0,0,0,0.2)',
                borderwidth=1,
                font=dict(size=12)
            ),
        ]
    )
    
    return fig

# Create and display the chart
fig = create_life_expectancy_chart()
fig.show()
